# Environment Setup

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load Model

In [2]:
model = load_model("traffic_sign_model.keras")

# Load Class Names

In [3]:
classes = {
    0:'Speed limit (20km/h)',
    1:'Speed limit (30km/h)',
    2:'Speed limit (50km/h)',
    3:'Speed limit (60km/h)',
    4:'Speed limit (70km/h)',
    5:'Speed limit (80km/h)',
    6:'End of speed limit (80km/h)',
    7:'Speed limit (100km/h)',
    8:'Speed limit (120km/h)',
    9:'No passing',
    10:'No passing veh over 3.5 tons',
    11:'Right-of-way at intersection',
    12:'Priority road',
    13:'Yield',
    14:'Stop',
    15:'No vehicles',
    16:'Veh > 3.5 tons prohibited',
    17:'No entry',
    18:'General caution',
    19:'Dangerous curve left',
    20:'Dangerous curve right',
    21:'Double curve',
    22:'Bumpy road',
    23:'Slippery road',
    24:'Road narrows on the right',
    25:'Road work',
    26:'Traffic signals',
    27:'Pedestrians',
    28:'Children crossing',
    29:'Bicycles crossing',
    30:'Beware of ice/snow',
    31:'Wild animals crossing',
    32:'End speed + passing limits',
    33:'Turn right ahead',
    34:'Turn left ahead',
    35:'Ahead only',
    36:'Go straight or right',
    37:'Go straight or left',
    38:'Keep right',
    39:'Keep left',
    40:'Roundabout mandatory',
    41:'End of no passing',
    42:'End no passing veh > 3.5 tons'
}

# Scan Video

In [4]:
video_path = "clip_segment.mp4"
cap = cv2.VideoCapture(video_path)

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps

print("FPS:", fps)
print("Frames:", frame_count)
print("Duration:", duration, "sec")

timestamps = []

frame_number = 0
SKIP = 5  # check every 5th frame

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_number % SKIP == 0:
        img = cv2.resize(frame, (224, 224))
        img = img.astype("float32") / 255.0
        img = np.expand_dims(img, axis=0)

        preds = model.predict(img, verbose=0)
        class_id = np.argmax(preds)
        confidence = preds[0][class_id]

        if confidence > 0.8:
            timestamps.append(frame_number / fps)

    frame_number += 1

cap.release()

print("✅ Finished scanning video")
print("Total detections found:", len(timestamps))

FPS: 30.0
Frames: 35249
Duration: 1174.9666666666667 sec
✅ Finished scanning video
Total detections found: 1822


# Filter Segment

In [5]:
segment_start = 50
segment_end = 1625

segment_detections = [t for t in timestamps if segment_start <= t <= segment_end]

print("\n✅ Detection Summary for Segment 14:10 → 33:45")
print("Detections inside segment:", len(segment_detections))

if segment_detections:
    print("\nFirst 10 timestamps:")
    print(segment_detections[:10])


✅ Detection Summary for Segment 14:10 → 33:45
Detections inside segment: 1583

First 10 timestamps:
[50.0, 50.166666666666664, 50.333333333333336, 50.5, 50.666666666666664, 50.833333333333336, 51.0, 51.166666666666664, 51.333333333333336, 51.5]


In [6]:
import cv2

# --- Input clip ---
input_video = "clip_segment.mp4"
output_video = "linkedin_demo.mp4"

# --- Load video ---
cap = cv2.VideoCapture(input_video)

fps = cap.get(cv2.CAP_PROP_FPS)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# --- Output writer ---
out = cv2.VideoWriter(
    output_video,
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (width, height)
)

# ✅ Your timestamps list (sorted)
detections = sorted(timestamps)      # <-- make sure 'timestamps' exists

i = 0  # detection counter index

print("✅ Processing video...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Current time position in seconds
    current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0

    # ✅ Count detections up to current frame
    while i < len(detections) and detections[i] <= current_time:
        i += 1

    # ✅ Add on-screen text
    cv2.putText(
        frame,
        f"Detections so far: {i}",
        (30, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.2,
        (255, 255, 255),
        3
    )

    out.write(frame)

cap.release()
out.release()

print("✅ Video created successfully!")
print("➡ Saved as:", output_video)


✅ Processing video...
✅ Video created successfully!
➡ Saved as: linkedin_demo.mp4


In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# --- Load your trained classifier ---
model = load_model("traffic_sign_model.keras")

# --- Class names ---
classes = {
    0:'Speed limit (20km/h)',
    1:'Speed limit (30km/h)',
    2:'Speed limit (50km/h)',
    3:'Speed limit (60km/h)',
    4:'Speed limit (70km/h)',
    5:'Speed limit (80km/h)',
    6:'End of speed limit (80km/h)',
    7:'Speed limit (100km/h)',
    8:'Speed limit (120km/h)',
    9:'No passing',
    10:'No passing veh over 3.5 tons',
    11:'Right-of-way at intersection',
    12:'Priority road',
    13:'Yield',
    14:'Stop',
    15:'No vehicles',
    16:'Veh > 3.5 tons prohibited',
    17:'No entry',
    18:'General caution',
    19:'Dangerous curve left',
    20:'Dangerous curve right',
    21:'Double curve',
    22:'Bumpy road',
    23:'Slippery road',
    24:'Road narrows on the right',
    25:'Road work',
    26:'Traffic signals',
    27:'Pedestrians',
    28:'Children crossing',
    29:'Bicycles crossing',
    30:'Beware of ice/snow',
    31:'Wild animals crossing',
    32:'End speed + passing limits',
    33:'Turn right ahead',
    34:'Turn left ahead',
    35:'Ahead only',
    36:'Go straight or right',
    37:'Go straight or left',
    38:'Keep right',
    39:'Keep left',
    40:'Roundabout mandatory',
    41:'End of no passing',
    42:'End no passing veh > 3.5 tons'
}

# --- Video setup ---
cap = cv2.VideoCapture("clip_segment.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("traffic_sign_detection_demo.mp4",
                      cv2.VideoWriter_fourcc(*"mp4v"),
                      fps,
                      (width, height))

IMG_SIZE = (224, 224)

print("✅ Processing video...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # --- Red color mask (traffic signs are often red-bordered) ---
    lower1 = np.array([0, 70, 50])
    upper1 = np.array([10, 255, 255])
    lower2 = np.array([170, 70, 50])
    upper2 = np.array([180, 255, 255])
    mask = cv2.inRange(hsv, lower1, upper1) | cv2.inRange(hsv, lower2, upper2)

    # --- Find contours (possible signs) ---
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        # Filter out too-small or too-large areas
        if w*h < 1000 or w*h > 50000:
            continue

        roi = frame[y:y+h, x:x+w]
        if roi.size == 0:
            continue

        # --- Classify the ROI ---
        img = cv2.resize(roi, IMG_SIZE)
        img = img.astype("float32") / 255.0
        img = np.expand_dims(img, axis=0)
        preds = model.predict(img, verbose=0)
        class_id = np.argmax(preds)
        conf = preds[0][class_id]

        if conf > 0.80:
            label = f"{classes[class_id]} ({conf*100:.1f}%)"
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, label, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
print("✅ Done! Video saved as traffic_sign_detection_demo.mp4")


✅ Processing video...
✅ Done! Video saved as traffic_sign_detection_demo.mp4
